<a href="https://colab.research.google.com/github/varlottaang/LangGraph/blob/main/YT_Adding_RAG_to_LangGraph_Agents_WestWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WestWorld Email Response

In [ ]:
!wget -q -O westworld_resort_facts.csv https://www.dropbox.com/scl/fi/qhzosgi21sqsymv5j4o1b/westworld_resort_facts.csv?rlkey=d81cez1bxck2y3lw53phar8nk&st=70bbkq9n&dl=1

In [ ]:
!pip -q install langchain-groq
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# for RAG Only
!pip -q install -U langchain langchain-community langchainhub
!pip -q install langchain-chroma bs4
!pip -q install huggingface_hub unstructured sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 48

# **The Agent's Goal**

Reply to a customer email
1. get the email
2. categorize the email as a "sales", "custom enquiry", "off topic", "customer complaint"
3. use the category & initial email to create questions for a RAG search to get info needed for the reply
4. write a draft reply
5. check the reply against the RAG answers
7. rewrite if needed



In [ ]:
import os
from pprint import pprint
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

## **Building the RAG**

### load sheet base

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader_csv = CSVLoader(file_path="/content/westworld_resort_facts.csv")

# data = loader_csv.load()
# print(data)

### Merge All Loader

In [ ]:
from langchain_community.document_loaders.merge import MergedDataLoader

loader_all = MergedDataLoader(loaders=[loader_csv]) #loader_web, loader_txt,]

In [ ]:
docs_all = loader_all.load()

In [ ]:
len(docs_all)

148

#### Text splitting if you have long documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs_all)

len(texts)

148

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma

persist_directory = 'db'

## Here you can change the embeddings etc
embedding = bge_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Testing the RAG

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [ ]:
#RAG Chain
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUESTION: {question} \n
    CONTEXT: {context} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)

rag_prompt_chain = rag_prompt | GROQ_LLM | StrOutputParser()

QUESTION = """What can I do in the Westworld Park?"""
CONTEXT = retriever.invoke(QUESTION)

result = rag_prompt_chain.invoke({"question": QUESTION, "context":CONTEXT})

print(result)

In Westworld Park, you can experience the Wild West in a realistic and interactive way, interacting with advanced androids called "hosts". You can also engage in various activities such as yoga, meditation, hiking, horseback riding, and visit saloons, theaters, museums, and workshops. Additionally, the park offers spa and wellness centers for relaxation and rejuvenation.


In [ ]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | rag_prompt
    | GROQ_LLM
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is the westworld park all about?")

'Westworld is an immersive, high-tech theme park that allows guests to experience the Wild West in a realistic and interactive way, populated by advanced androids called "hosts". It offers a unique and immersive experience that blurs the line between fantasy and reality, allowing guests to become active participants in their own stories. The park\'s advanced AI hosts and meticulously crafted environments create experiences that are deeply personal and emotionally engaging.'

In [ ]:
rag_chain.invoke("who are some of the hosts I can meet?")

'You can meet and interact with hosts based on Maeve, Dolores, Elsie, Clementine, and Charlotte Hale in the park. These hosts are available for conversations, storylines, and adventures.'

In [ ]:
rag_chain.invoke("Who is Ford?")

"Dr. Ford is a key figure in the operations of Westworld, an immersive theme park. He is not a character that guests can meet directly, but they can engage with a host designed to discuss the history and philosophy behind Westworld's creation, offering insights similar to those of Dr. Ford."

In [ ]:
rag_chain.invoke("Can I sleep under the stars there?")

'It is unclear if sleeping under the stars is possible in Westworld. The context does not provide specific information about overnight stays or camping in the park.'

## Making the Agent

In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

## Utils

In [ ]:

def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  if type(content) == dict:
    content = '\n'.join(f"{key}: {value}" for key, value in content.items())
  if type(content) == list:
    content = '\n'.join(content)
  with open(f"{filename}.md", "w") as f:
    f.write(content)


## Basic Chains

1. Categorize EMAIL  
2. Research Router  # Not using for now
3. RAG Questions
4. Write Draft Email  
5. Rewrite Router  
6. Draft Email Analysis  
7. Rewrite Email
8. RAG Chain - created in RAG section above

In [ ]:
#Categorize EMAIL
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Categorizer Agent for the theme park Westworld,You are a master at \
    understanding what a customer wants when they write an email and are able to categorize \
    it in a useful way. Remember people maybe asking about experiences they can have in westworld. \

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        price_equiry - used when someone is asking for information about pricing \
        customer_complaint - used when someone is complaining about something \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        customer_feedback - used when someone is giving feedback about a product \
        off_topic when it doesnt relate to any other category \


            Output a single cetgory only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
I am emailing to find out info about your them park and what I can do there. \n

I am looking for new experiences.

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

'product_enquiry'


In [ ]:
## Research Router
research_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the initial email and routing to our internal knowledge system\
     or directly to a draft email. \n

    Use the following criteria to decide how to route the email: \n\n

    If the initial email only requires a simple response
    Just choose 'draft_email'  for questions you can easily answer, prompt engineering, and adversarial attacks.
    If the email is just saying thank you etc then choose 'draft_email'

    If you are unsure or the person is asking a question you don't understand then choose 'research_info'

    You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use research-info.
    Give a binary choice 'research_info' or 'draft_email' based on the question. Return the a JSON with a single key 'router_decision' and
    no premable or explaination. use both the initial email and the email category to make your decision
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Email to route INITIAL_EMAIL : {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

research_router = research_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'product_enquiry'

print(research_router.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'router_decision': 'research_info'}


In [ ]:
## RAG QUESTIONS
search_rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best questions to ask our knowledge agent to get the best info for the customer.

    given the INITIAL_EMAIL and EMAIL_CATEGORY. Work out the best questions that will find the best \
    info for helping to write the final email. Remember when people ask about a generic park they are \
    probably reffering to the park WestWorld. Write the questions to our knowledge system not to the customer.

    Return a JSON with a single key 'questions' with no more than 3 strings of and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

question_rag_chain = search_rag_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'product_enquiry'
research_info = None

print(question_rag_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'questions': ['What are the main attractions and experiences offered at WestWorld theme park?', 'What are some new or unique experiences that WestWorld theme park has to offer?', 'What are the most popular activities or rides at WestWorld theme park?']}


In [ ]:
## Write Draft Email
draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent for the theme park Westworld, take the INITIAL_EMAIL below \
    from a human that has emailed our company email address, the email_category \
    that the categorizer agent gave it and the research from the research agent and \
    write a helpful email in a thoughtful and friendly way. Remember people maybe asking \
    about experiences they can have in westworld.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information that hasn't been provided by the research_info or in the initial_email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category,"research_info":research_info}))

{'email_draft': "Dear Paul,\n\nThank you for reaching out to us! We're thrilled to hear that you're interested in exploring new experiences at Westworld. Our theme park offers a wide range of immersive experiences that will transport you to a world of wonder and adventure.\n\nFrom exploring the vast open plains on horseback to engaging in high-stakes shootouts, we have something for every thrill-seeker. You can also indulge in luxurious accommodations, savor delicious cuisine, and interact with our lifelike hosts.\n\nIf you're looking for something more, we have a variety of unique experiences tailored to your interests. Would you be interested in learning more about our special packages or customized itineraries?\n\nPlease let us know, and we'll be happy to help you plan an unforgettable adventure at Westworld.\n\nBest regards,\nSarah\nResident Manager"}


In [ ]:
## Rewrite Router
rewrite_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at evaluating the emails that are draft emails for the customer and deciding if they
    need to be rewritten to be better. \n

    Use the following criteria to decide if the DRAFT_EMAIL needs to be rewritten: \n\n

    If the INITIAL_EMAIL only requires a simple response which the DRAFT_EMAIL contains then it doesn't need to be rewritten.
    If the DRAFT_EMAIL addresses all the concerns of the INITIAL_EMAIL then it doesn't need to be rewritten.
    If the DRAFT_EMAIL is missing information that the INITIAL_EMAIL requires then it needs to be rewritten.

    Give a binary choice 'rewrite' (for needs to be rewritten) or 'no_rewrite' (for doesn't need to be rewritten) based on the DRAFT_EMAIL and the criteria.
    Return the a JSON with a single key 'router_decision' and no premable or explaination. \
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    DRAFT_EMAIL: {draft_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","draft_email"],
)

rewrite_router = rewrite_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
draft_email = "Yo we can't help you, best regards Sarah"

print(rewrite_router.invoke({"initial_email": EMAIL, "email_category":email_category, "draft_email":draft_email}))

{'router_decision': 'rewrite'}


In [ ]:
## Draft Email Analysis
draft_analysis_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Quality Control Agent read the INITIAL_EMAIL below  from a human that has emailed \
    our company email address, the email_category that the categorizer agent gave it and the \
    research from the research agent and write an analysis of how the email.

    Check if the DRAFT_EMAIL addresses the customer's issued based on the email category and the \
    content of the initial email.\n

    Give feedback of how the email can be improved and what specific things can be added or change\
    to make the email more effective at addressing the customer's issues.

    You never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Return the analysis a JSON with a single key 'draft_analysis' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n\n
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_analysis_chain = draft_analysis_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

email_analysis = draft_analysis_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email})

pprint(email_analysis)

{'draft_analysis': {'addresses_customer_issue': False,
                    'feedback': {'changes_needed': ["Respond to the customer's "
                                                    'inquiry about the theme '
                                                    'park and experiences.',
                                                    'Use a professional and '
                                                    'friendly tone.',
                                                    'Provide helpful '
                                                    'information or '
                                                    'alternatives to the '
                                                    'customer.'],
                                 'improvement_suggestions': ['The draft email '
                                                             'does not address '
                                                             "the customer's "
                                      

In [ ]:
# Rewrite Email with Analysis
rewrite_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Final Email Agent read the email analysis below from the QC Agent \
    and use it to rewrite and improve the draft_email to create a final email.


    You never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Return the final email as JSON with a single key 'final_email' which is a string and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    DRAFT_EMAIL_FEEDBACK: {email_analysis} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email",
                     "email_category",
                     "research_info",
                     "email_analysis",
                     "draft_email",
                     ],
)

rewrite_chain = rewrite_email_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

final_email = rewrite_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email,
                                "email_analysis":email_analysis})

final_email['final_email']

"Dear valued customer,\n\nI apologize for any inconvenience you've experienced. I'm happy to help you with your inquiry about the theme park and experiences. Unfortunately, I don't have enough information to provide a specific answer to your question. Could you please provide more details about your concern? I'll do my best to assist you.\n\nThank you for your patience and understanding.\n\nBest regards,\nSarah"

## State

In [ ]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [ ]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category
        draft_email: LLM generation
        final_email: LLM generation
        research_info: list of documents
        info_needed: whether to add search info
        num_steps: number of steps
    """
    initial_email : str
    email_category : str
    draft_email : str
    final_email : str
    research_info : List[str] # this will now be the RAG results
    info_needed : bool
    num_steps : int
    draft_email_feedback : dict
    rag_questions : List[str]

## Nodes

1. categorize_email
2. research_info_search # now done using RAG
3. draft_email_writer  
4. analyze_draft_email  
5. rewrite_email  
6. no_rewrite  
7. state_printer


In [ ]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [ ]:

def research_info_search(state):

    print("---RESEARCH INFO RAG---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    questions = question_rag_chain.invoke({"initial_email": initial_email,
                                            "email_category": email_category })
    questions = questions['questions']
    # print(questions)
    rag_results = []
    for question in questions:
        print(question)
        temp_docs = rag_chain.invoke(question)
        print(temp_docs)
        question_results = question + '\n\n' + temp_docs + "\n\n\n"
        if rag_results is not None:
            rag_results.append(question_results)
        else:
            rag_results = [question_results]
    print(rag_results)
    print(type(rag_results))
    write_markdown_file(rag_results, "research_info")
    write_markdown_file(questions, "rag_questions")
    return {"research_info": rag_results,"rag_questions":questions, "num_steps":num_steps}

In [ ]:
def draft_email_writer(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info":research_info})
    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [ ]:
def analyze_draft_email(state):
    print("---DRAFT EMAIL ANALYZER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email_feedback = draft_analysis_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email}
                                               )
    # print(draft_email)
    # print(type(draft_email))

    write_markdown_file(str(draft_email_feedback), "draft_email_feedback")
    return {"draft_email_feedback": draft_email_feedback, "num_steps":num_steps}

In [ ]:
def rewrite_email(state):
    print("---ReWRITE EMAIL ---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    draft_email_feedback = state["draft_email_feedback"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    final_email = rewrite_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email,
                                                "email_analysis": draft_email_feedback}
                                               )

    write_markdown_file(str(final_email), "final_email")
    return {"final_email": final_email['final_email'], "num_steps":num_steps}

In [ ]:
def no_rewrite(state):
    print("---NO REWRITE EMAIL ---")
    ## Get the state
    draft_email = state["draft_email"]
    num_steps = state['num_steps']
    num_steps += 1

    write_markdown_file(str(draft_email), "final_email")
    return {"final_email": draft_email, "num_steps":num_steps}

In [ ]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Draft Email: {state['draft_email']} \n" )
    print(f"Final Email: {state['final_email']} \n" )
    print(f"Research Info: {state['research_info']} \n")
    print(f"RAG Questions: {state['rag_questions']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

## Conditional Edges

In [ ]:
def route_to_research(state):
    """
    Route email to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]


    router = research_router.invoke({"initial_email": initial_email,"email_category":email_category })
    print(router)
    # print(type(router))
    print(router['router_decision'])
    if router['router_decision'] == 'research_info':
        print("---ROUTE EMAIL TO RESEARCH INFO---")
        return "research_info"
    elif router['router_decision'] == 'draft_email':
        print("---ROUTE EMAIL TO DRAFT EMAIL---")
        return "draft_email"

In [ ]:
def route_to_rewrite(state):

    print("---ROUTE TO REWRITE---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    # research_info = state["research_info"]

    # draft_email = "Yo we can't help you, best regards Sarah"

    router = rewrite_router.invoke({"initial_email": initial_email,
                                     "email_category":email_category,
                                     "draft_email":draft_email}
                                   )
    print(router)
    print(router['router_decision'])
    if router['router_decision'] == 'rewrite':
        print("---ROUTE TO ANALYSIS - REWRITE---")
        return "rewrite"
    elif router['router_decision'] == 'no_rewrite':
        print("---ROUTE EMAIL TO FINAL EMAIL---")
        return "no_rewrite"

## Build the Graph

### Add Nodes

In [ ]:

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("categorize_email", categorize_email) # categorize email
workflow.add_node("research_info_search", research_info_search) # web search
workflow.add_node("state_printer", state_printer)
workflow.add_node("draft_email_writer", draft_email_writer)
workflow.add_node("analyze_draft_email", analyze_draft_email)
workflow.add_node("rewrite_email", rewrite_email)
workflow.add_node("no_rewrite", no_rewrite)



### Add Edges

In [ ]:
workflow.set_entry_point("categorize_email")

# workflow.add_conditional_edges(
#     "categorize_email",
#     route_to_research,
#     {
#         "research_info": "research_info_search",
#         "draft_email": "draft_email_writer",
#     },
# )
workflow.add_edge("categorize_email", "research_info_search")
workflow.add_edge("research_info_search", "draft_email_writer")


workflow.add_conditional_edges(
    "draft_email_writer",
    route_to_rewrite,
    {
        "rewrite": "analyze_draft_email",
        "no_rewrite": "no_rewrite",
    },
)
workflow.add_edge("analyze_draft_email", "rewrite_email")
workflow.add_edge("no_rewrite", "state_printer")
workflow.add_edge("rewrite_email", "state_printer")
workflow.add_edge("state_printer", END)

In [ ]:
# Compile
app = workflow.compile()

In [ ]:
# EMAIL = """HI there, \n
# I am emailing to find out the current price of Bitcoin. \n

# Can you please help me/

# Thanks,
# John
# """

EMAIL = """HI there, \n
I am emailing to find out info about your them park and what I can do there. \n

I am looking for new experiences.

Thanks,
Paul
"""


EMAIL = """HI there, \n
I am a big fan of westworld.
can I meet Maeve in the park? Really want to chat with her.

Thanks,
Ringo
"""

In [ ]:
# run the agent
inputs = {"initial_email": EMAIL, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
product_enquiry
'Finished running: categorize_email:'
---RESEARCH INFO RAG---
What are the character meet and greet options available in WestWorld?
In WestWorld, guests can meet and interact with hosts modeled after Dolores, Maeve, Clementine, Logan, and Teddy. Each character offers unique experiences, such as going on an adventure with Dolores, engaging in conversations with Maeve, or training with Teddy.
Are there any special experiences or tours that allow interaction with WestWorld characters like Maeve?
Yes, guests can meet and interact with a host based on Maeve, engaging in conversations and storylines that explore her wit, intelligence, and growing self-awareness. You can also embark on a thrilling heist storyline with a host modeled after Hector, which involves working with Maeve. Additionally, you can explore romantic storylines with Maeve or other hosts of your choice.
Can guests have personalized conversations with WestWorld characters like 

In [ ]:
output = app.invoke(inputs)

print(output['final_email'])

---CATEGORIZING INITIAL EMAIL---
'product_enquiry'
---RESEARCH INFO RAG---
Is Maeve a character that can be met in WestWorld park?
Yes, Maeve is a host that guests can meet and interact with in Westworld park. You can engage in conversations and storylines that explore her wit, intelligence, and growing self-awareness.
Are character meet-and-greets available in WestWorld?
Yes, character meet-and-greets are available in WestWorld. Guests can request to meet and interact with hosts modeled after various characters, including Dolores, Maeve, Clementine, and Logan. These interactions can involve conversations, adventures, and exploring storylines with the hosts.
What are the possibilities of interacting with WestWorld characters?
Guests can interact with Westworld characters, such as Maeve, Dolores, and Hector, through conversations, storylines, and adventures. Intimate interactions are also possible, but must be consensual and adhere to the park's ethical guidelines. Additionally, guests 

In [ ]:
!cat /content/email_category.md

'product_enquiry'

In [ ]:
!cat /content/draft_email.md

Dear Ringo,

Thank you for reaching out to us! We're thrilled to hear that you're a big fan of Westworld. 

I'm happy to inform you that yes, you can meet Maeve in the park! Maeve is a host that guests can meet and interact with in Westworld park. You can engage in conversations and storylines that explore her wit, intelligence, and growing self-awareness.

You can request to meet and interact with Maeve, and even engage in conversations, adventures, and exploring storylines with her. Our hosts, including Maeve, are designed to provide an immersive and interactive experience for our guests.

We hope you enjoy your time in Westworld and your interaction with Maeve!

Best regards,
Sarah
Resident Manager

In [ ]:
!cat /content/research_info.md

Is Maeve a character that can be met in WestWorld park?

Yes, Maeve is a host that guests can meet and interact with in Westworld park. You can engage in conversations and storylines that explore her wit, intelligence, and growing self-awareness.



Are character meet-and-greets available in WestWorld?

Yes, character meet-and-greets are available in WestWorld. Guests can request to meet and interact with hosts modeled after various characters, including Dolores, Maeve, Clementine, and Logan. These interactions can involve conversations, adventures, and exploring storylines with the hosts.



What are the possibilities of interacting with WestWorld characters?

Guests can interact with Westworld characters, such as Maeve, Dolores, and Hector, through conversations, storylines, and adventures. Intimate interactions are also possible, but must be consensual and adhere to the park's ethical guidelines. Additionally, guests can interact with each other within the park, and many storylines 

In [ ]:
!cat /content/rag_questions.md

Is Maeve a character that can be met in WestWorld park?
Are character meet-and-greets available in WestWorld?
What are the possibilities of interacting with WestWorld characters?